In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from PIL import Image
from skimage.segmentation import mark_boundaries

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

root_dir = "../"
data_dir = os.path.join(root_dir, "data")
model_dir = os.path.join(root_dir, "pretrained_model")
explanation_dir = os.path.join(root_dir, "explanations")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

model = torch.hub.load("pytorch/vision:v0.10.0", "resnet18", pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)
model.load_state_dict(
    torch.load(os.path.join(model_dir, "model.pt"), map_location=device)
)
model.to(device)
model.eval()
x = torch.randn(1, 3, 1200, 1600, device=device)
model(x)
torch.cuda.empty_cache()

mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)]
)

In [ ]:
limexp = lime_image.LimeImageExplainer()


def f(x):
    x = torch.stack(tuple(transform(i) for i in x), dim=0)
    x = x.to(device)
    output = model(x)
    p = F.softmax(output, dim=1)
    return p.detach().cpu().numpy()

In [ ]:
true_positives = np.load(os.path.join(explanation_dir, "true_positive.npy"))

for i, image_path in enumerate(true_positives[[0]]):
    image_name = os.path.basename(image_path).split(".")[0]
    image = Image.open(os.path.join(root_dir, image_path))
    image_rgb = image.convert("RGB")

    image = np.array(image_rgb)
    segmentation_fn = SegmentationAlgorithm(
        "quickshift", kernel_size=4, max_dist=200, ratio=0.2
    )
    segments = segmentation_fn(image)
    plt.imshow(mark_boundaries(image, segments))
    plt.show()

    explanation = limexp.explain_instance(
        image, f, top_labels=1, num_samples=200, segmentation_fn=segmentation_fn
    )
    _, explanation = explanation.get_image_and_mask(
        explanation.top_labels[0], positive_only=True, num_features=50, hide_rest=False
    )
    _max = np.amax(np.abs(explanation.flatten()))
    plt.imshow(image)
    plt.imshow(explanation, cmap="bwr", alpha=0.5, vmin=-_max, vmax=_max)
    plt.show()
    plt.imshow(explanation > 0, cmap="bwr", alpha=0.5)
    plt.show()
    break